In [1]:
from typing import List


def merge_consecutive_thought_token_indexes(thought_token_indexes: List[int]) -> List[int]:
    new_indexes = []
    
    for i in range(len(thought_token_indexes) - 1):
        this_token = thought_token_indexes[i]
        next_token = thought_token_indexes[i + 1]

        if this_token + 1 != next_token and this_token != next_token:
            new_indexes.append(this_token)

    new_indexes.append(thought_token_indexes[-1])

    return new_indexes



merge_consecutive_thought_token_indexes(
    [2,3]
)

import torch
thought_token_index = merge_consecutive_thought_token_indexes(
    [1,2,3,7,8,9,20,20]
) # expect 3,9,20

bottom = torch.ones(20, dtype=torch.long) * (20 - 1)

for i in range(20):
    bottom[i] = thought_token_index[0]
    if thought_token_index[0] == i:
        thought_token_index.pop(0)


In [19]:
def bounds_to_mask(top, bottom, max_seq_len=16):
    batch_size = top.size(0)
    range_tensor = torch.arange(max_seq_len, device=top.device).view(1, -1, 1).expand(batch_size, -1, -1)
    # Create a mask for values greater than or equal to top
    top_mask = range_tensor >= top.view(batch_size, 1, -1)

    # Create a mask for values less than or equal to bottom
    bottom_mask = range_tensor <= bottom.view(batch_size, 1, -1)
    
    # Combine the masks and convert to long
    mask = (top_mask & bottom_mask)
    
    return mask


# set torch output to large
torch.set_printoptions(linewidth=200)
top = torch.arange(20, dtype=torch.long)
bounds_to_mask(top.unsqueeze(0), bottom.unsqueeze(0), max_seq_len=20).to(torch.long)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
      

In [14]:
def bounds_to_mask(top, bottom, max_seq_len=16):
    batch_size = top.size(0)
    range_tensor = torch.arange(max_seq_len, device=top.device).view(1, -1, 1).expand(batch_size, -1, -1)
    # Create a mask for values greater than or equal to top
    top_mask = range_tensor >= top.view(batch_size, 1, -1)

    # Create a mask for values less than or equal to bottom
    bottom_mask = range_tensor <= bottom.view(batch_size, 1, -1)
    
    # Combine the masks and convert to long
    mask = (top_mask & bottom_mask)
    
    return mask


# set torch output to large
torch.set_printoptions(linewidth=200)
top = torch.arange(20, dtype=torch.long)
bottom = torch.ones(20, dtype=torch.long) * (20 - 1)
bounds_to_mask(top.unsqueeze(0), bottom.unsqueeze(0), max_seq_len=20).to(torch.long)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
      

In [20]:
def bounds_to_mask(top, bottom, max_seq_len=16):
    batch_size = top.size(0)
    range_tensor = torch.arange(max_seq_len, device=top.device).view(1, -1, 1).expand(batch_size, -1, -1)
    # Create a mask for values greater than or equal to top
    top_mask = range_tensor >= top.view(batch_size, 1, -1)

    # Create a mask for values less than or equal to bottom
    bottom_mask = range_tensor <= bottom.view(batch_size, 1, -1)
    
    # Combine the masks and convert to long
    mask = (top_mask & bottom_mask)
    
    return mask


# set torch output to large
torch.set_printoptions(linewidth=200)
top = torch.arange(20, dtype=torch.long)

# [1,2,3,7,8,9,20,20]
bottom = torch.tensor([
    3,9,9,9,9,
    9,9,20,20,20,
    20,20,20,20,20,
    20,20,20,20,20,
]) # EXPECTED I THINK
bounds_to_mask(top.unsqueeze(0), bottom.unsqueeze(0), max_seq_len=20).to(torch.long)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
      

In [ ]:
from typing import List

def group_thought_tokens(thought_token_indexes: List[int]) -> List[List[int]]:
    groups = []

    group = []
    for i in range(len(thought_token_indexes) - 1):
        group.append(thought_token_indexes[i])
        
        if thought_token_indexes[i+1] != thought_token_indexes[i] + 1:
            groups.append(group)
            group = []
    
    return groups



merge_consecutive_thought_token_indexes(
    [2,3]
)

import torch
thought_token_index = merge_consecutive_thought_token_indexes(
    [1,2,3,7,8,9,20,20]
) # expect 3,9,20

bottom = torch.ones(20, dtype=torch.long) * (20 - 1)

for i in range(20):
    bottom[i] = thought_token_index[0]
    if thought_token_index[0] == i:
        thought_token_index.pop(0)


In [16]:
thought_token_indexes = [1,2,3,7,8,9,19]

groups = []
group = []
for i in range(len(thought_token_indexes)):
    group.append(thought_token_indexes[i])
    
    if i == len(thought_token_indexes) - 1 or thought_token_indexes[i+1] != thought_token_indexes[i] + 1:
        groups.append(group)
        group = []

groups

[[1, 2, 3], [7, 8, 9], [19]]

In [17]:
bottom = torch.ones(20, dtype=torch.long) * (20 - 1)

for i in range(20):
    if i in groups[0]:
        groups.pop(0)
    if len(groups) == 0:
        break
    bottom[i] = groups[0][-1]
        
bottom

tensor([ 3,  9,  9,  9,  9,  9,  9, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19])

In [19]:
i1 = [1.0, 0.9583333134651184, 0.9166666865348816, 0.875, 0.8333333134651184, 0.7916666865348816, 0.75, 0.7083333134651184, 0.6666666865348816, 0.625, 0.5833333134651184, 0.5416666865348816, 0.5, 0.4583333432674408, 0.4166666567325592, 0.375, 0.3333333432674408, 0.2916666567325592, 0.25, 0.2083333283662796, 0.1666666716337204, 0.125, 0.0833333358168602, 0.0416666679084301]
c1 = [0.0007057780749164522, 0.0009062369354069233, 0.0011636312119662762, 0.0014941320987418294, 0.0019185036653652787, 0.0024634075816720724, 0.00316307763569057, 0.0040614721365273, 0.0052150338888168335, 0.006696235854178667, 0.008598136715590954, 0.011040225625038147, 0.014175930060446262, 0.01820225641131401, 0.023372158408164978, 0.030010445043444633, 0.038534171879291534, 0.049478862434625626, 0.06353211402893066, 0.0815768614411354, 0.1047467514872551, 0.1344974935054779, 0.17269819974899292, 0.22174887359142303]


In [20]:
i2 = [0.9473673105239868, 0.9059182405471802, 0.863993227481842, 0.822013258934021, 0.7806047797203064, 0.7390885949134827, 0.6977716088294983, 0.6565199494361877, 0.6153582334518433, 0.5742072463035583, 0.533133327960968, 0.4919048249721527, 0.45102840662002563, 0.4101402163505554, 0.3696402907371521, 0.3284718692302704, 0.288100928068161, 0.24667084217071533, 0.20608770847320557, 0.16387209296226501, 0.12154611945152283, 0.08005927503108978, 0.03694465756416321, 0.0]
c2 = [0.0007593068294227123, 0.0009701819508336484, 0.0012388600734993815, 0.001593381748534739, 0.002042352920398116, 0.0026283820625394583, 0.0033526336774230003, 0.00429035909473896, 0.005484136752784252, 0.006990407593548298, 0.008920209482312202, 0.011328067630529404, 0.014373554848134518, 0.01835733838379383, 0.022799434140324593, 0.02904142439365387, 0.03722739964723587, 0.04783306270837784, 0.0611923448741436, 0.0783417671918869, 0.1005365327000618, 0.12955178320407867, 0.18052154779434204, 0.23062551021575928]


In [21]:
import numpy

print(numpy.array(i1) - numpy.array(i2))
print(numpy.array(c1) - numpy.array(c2))

[0.05263269 0.05241507 0.05267346 0.05298674 0.05272853 0.05257809
 0.05222839 0.05181336 0.05130845 0.05079275 0.05019999 0.04976186
 0.04897159 0.04819313 0.04702637 0.04652813 0.04523242 0.04499581
 0.04391229 0.04446124 0.04512055 0.04494072 0.04638868 0.04166667]
[-5.35287545e-05 -6.39450154e-05 -7.52288615e-05 -9.92496498e-05
 -1.23849255e-04 -1.64974481e-04 -1.89556042e-04 -2.28886958e-04
 -2.69102864e-04 -2.94171739e-04 -3.22072767e-04 -2.87842005e-04
 -1.97624788e-04 -1.55081972e-04  5.72724268e-04  9.69020650e-04
  1.30677223e-03  1.64579973e-03  2.33976915e-03  3.23509425e-03
  4.21021879e-03  4.94571030e-03 -7.82334805e-03 -8.87663662e-03]
